# Partie prédictive:
## Vous mettrez en place un modèle de regression linéaire qui permet d'estimer le temps de trajet en fonction de sa distance.

In [1]:
import pandas as pd
import sys

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

sys.path.append("../NYC-Taxi")

df = pd.read_csv("../data/02_intermediate/train.csv")
#df = pd.read_csv("../data/02_intermediate/dirty_train.csv")

#### Définitions de définitions pour récupérer les indicateurs et formater leur affichage
(Les fonctions sont définies ici et non dans un fichier à part à cause d'un soucis d'import de modules)

In [2]:
def get_model_metrics(model, X, y):
    """
        Separate data in train and test sets,
        fit the model,
        make predictions on train and test datas,
        print metrics

        params:
            model(function): model used with params (ie: Lasso(alpha=x)) 
            X(DataFrame): DataFrame subset with selected features,
            y(Series): variable to predict

        returns: 
                print β1, β0, R2 and RMSE
    """
    # Split des datas
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=0.8, random_state=1
    )
    model.fit(X_train, y_train)
    # Affichage des β1 pour chaque variable
    for idx, name in enumerate(X_train.columns):
        print(f"β1 de {name} : {round(model.coef_[idx], 3)}")

    print(f"β0 (intercept_) : {round(model.intercept_, 3)}\n")

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    y_list = [y_train, y_train_pred, y_test, y_test_pred]
    get_r2_rmse(y_list)


def get_r2_rmse(y_list):
    """
        calculate R2 and RMSE for each sets (train and test)
        and format output

        param:
            y_list = [
                y_train, 
                y_train_pred, 
                y_test, 
                y_test_pred
            ]
    """

    sets = ["Training", "Testing "]
    i = 0
    for set in sets:
        r2 = round(r2_score(y_list[i], y_list[i+1]), 3)
        rmse = round(mean_squared_error(y_list[i], y_list[i+1], squared=False), 3)
        print(
            "{} set : R2 = {}, RMSE = {}".format(set, r2, rmse)
        )
        i += 2

### Régression linéaire à un paramètre

In [3]:
y = df.trip_duration
df_features = ["distance_km"]
X = df[df_features]

In [4]:
get_model_metrics(LinearRegression(), X, y)


β1 de distance_km : 139.341
β0 (intercept_) : 367.681

Training set : R2 = 0.493, RMSE = 445.235
Testing  set : R2 = 0.496, RMSE = 439.209


## Modèle multi linéaire
### Modèle 1
#### Hypothèse à vérifier : absence de colinéarité

In [5]:
df[[
    "trip_duration",
    "distance_km",
    "pickup_dayofweek",
    "pickup_part4h"
]].corr()


,trip_duration,distance_km,pickup_dayofweek,pickup_part4h
trip_duration,1.000000,0.702478,-0.032675,0.029935
distance_km,0.702478,1.000000,0.013581,-0.024856
pickup_dayofweek,-0.032675,0.013581,1.000000,-0.085316
pickup_part4h,0.029935,-0.024856,-0.085316,1.000000


La matrix de corrélation ci-dessus nous apprend que :
- Les coefficients de corrélation sont inférieur à 0.8.
- La distance est la variable la plus corrélé, tandis que que les deux autres variables ont un impacts assez faible
- **Hypothèse vérifiée** : absence de colinéarité entre les variables explicatives

### Choix des features :
- *distance* : indicateur pincipale pour calculer la durée du trajet
- *pickup_dayofweek* : on suppose que le jour à une influence sur l'état du trafic et donc sur la durée du trajet
- *pickup_part4h* :on suppose que le moment de la journée à une influence sur l'état du trafic et donc sur la durée du trajet

In [6]:
y = df.trip_duration
df_features = [
    "distance_km",
    "pickup_dayofweek",
    "pickup_part4h"
]
X = df[df_features]

## Évaluation du modèle
### LinearRegression()

In [7]:
get_model_metrics(LinearRegression(), X, y)


β1 de distance_km : 139.664
β1 de pickup_dayofweek : -12.314
β1 de pickup_part4h : 17.583
β0 (intercept_) : 351.128

Training set : R2 = 0.497, RMSE = 443.596
Testing  set : R2 = 0.499, RMSE = 437.603


#### Liste des alpha à utiliser pour les Lasso() et les Ridge()

In [8]:
alphas = [0.1*10**(i) for i in range(4)]

### Lasso() avec différents alpha

In [9]:
for a in alphas:
    print(f"Avec Alpha = {a}")
    get_model_metrics(Lasso(alpha=a), X, y)
    print("\n")


Avec Alpha = 0.1
β1 de distance_km : 139.653
β1 de pickup_dayofweek : -12.29
β1 de pickup_part4h : 17.544
β0 (intercept_) : 351.204

Training set : R2 = 0.497, RMSE = 443.597
Testing  set : R2 = 0.499, RMSE = 437.602


Avec Alpha = 1.0
β1 de distance_km : 139.557
β1 de pickup_dayofweek : -12.075
β1 de pickup_part4h : 17.199
β0 (intercept_) : 351.896

Training set : R2 = 0.497, RMSE = 443.597
Testing  set : R2 = 0.499, RMSE = 437.601


Avec Alpha = 10.0
β1 de distance_km : 138.589
β1 de pickup_dayofweek : -9.93
β1 de pickup_part4h : 13.745
β0 (intercept_) : 358.816

Training set : R2 = 0.496, RMSE = 443.679
Testing  set : R2 = 0.499, RMSE = 437.658


Avec Alpha = 100.0
β1 de distance_km : 129.265
β1 de pickup_dayofweek : -0.0
β1 de pickup_part4h : 0.0
β0 (intercept_) : 399.222

Training set : R2 = 0.49, RMSE = 446.365
Testing  set : R2 = 0.493, RMSE = 440.188




### Ridge() avec différents alpha

In [10]:
for a in alphas:
    print(f"Avec Alpha = {a}")
    get_model_metrics(Ridge(alpha=a), X, y)
    print("\n")

Avec Alpha = 0.1
β1 de distance_km : 139.664
β1 de pickup_dayofweek : -12.314
β1 de pickup_part4h : 17.583
β0 (intercept_) : 351.128

Training set : R2 = 0.497, RMSE = 443.596
Testing  set : R2 = 0.499, RMSE = 437.603


Avec Alpha = 1.0
β1 de distance_km : 139.664
β1 de pickup_dayofweek : -12.314
β1 de pickup_part4h : 17.583
β0 (intercept_) : 351.128

Training set : R2 = 0.497, RMSE = 443.596
Testing  set : R2 = 0.499, RMSE = 437.603


Avec Alpha = 10.0
β1 de distance_km : 139.664
β1 de pickup_dayofweek : -12.314
β1 de pickup_part4h : 17.583
β0 (intercept_) : 351.128

Training set : R2 = 0.497, RMSE = 443.596
Testing  set : R2 = 0.499, RMSE = 437.603


Avec Alpha = 100.0
β1 de distance_km : 139.663
β1 de pickup_dayofweek : -12.314
β1 de pickup_part4h : 17.582
β0 (intercept_) : 351.133

Training set : R2 = 0.497, RMSE = 443.596
Testing  set : R2 = 0.499, RMSE = 437.603




 # Modèle 2

In [11]:
y = df.trip_duration
df_features = [
    "distance_km",
    "pickup_dayofweek"
]
X = df[df_features]

## Évaluation du modèle
### LinearRegression()

In [12]:
get_model_metrics(LinearRegression(), X, y)

β1 de distance_km : 139.456
β1 de pickup_dayofweek : -13.514
β0 (intercept_) : 408.545

Training set : R2 = 0.495, RMSE = 444.452
Testing  set : R2 = 0.497, RMSE = 438.43


### Lasso() avec différents alpha

In [13]:
for a in alphas:
    print(f"Avec Alpha = {a}")
    get_model_metrics(Lasso(alpha=a), X, y)
    print("\n")


Avec Alpha = 0.1
β1 de distance_km : 139.445
β1 de pickup_dayofweek : -13.487
β0 (intercept_) : 408.496

Training set : R2 = 0.495, RMSE = 444.452
Testing  set : R2 = 0.497, RMSE = 438.429


Avec Alpha = 1.0
β1 de distance_km : 139.353
β1 de pickup_dayofweek : -13.249
β0 (intercept_) : 408.06

Training set : R2 = 0.495, RMSE = 444.453
Testing  set : R2 = 0.498, RMSE = 438.428


Avec Alpha = 10.0
β1 de distance_km : 138.426
β1 de pickup_dayofweek : -10.869
β0 (intercept_) : 403.7

Training set : R2 = 0.495, RMSE = 444.494
Testing  set : R2 = 0.497, RMSE = 438.453


Avec Alpha = 100.0
β1 de distance_km : 129.265
β1 de pickup_dayofweek : -0.0
β0 (intercept_) : 399.222

Training set : R2 = 0.49, RMSE = 446.365
Testing  set : R2 = 0.493, RMSE = 440.188




### Ridge() avec différents alpha

In [14]:
for a in alphas:
    print(f"Avec Alpha = {a}")
    get_model_metrics(Ridge(alpha=a), X, y)
    print("\n")

Avec Alpha = 0.1
β1 de distance_km : 139.456
β1 de pickup_dayofweek : -13.514
β0 (intercept_) : 408.545

Training set : R2 = 0.495, RMSE = 444.452
Testing  set : R2 = 0.497, RMSE = 438.43


Avec Alpha = 1.0
β1 de distance_km : 139.456
β1 de pickup_dayofweek : -13.514
β0 (intercept_) : 408.545

Training set : R2 = 0.495, RMSE = 444.452
Testing  set : R2 = 0.497, RMSE = 438.43


Avec Alpha = 10.0
β1 de distance_km : 139.456
β1 de pickup_dayofweek : -13.514
β0 (intercept_) : 408.545

Training set : R2 = 0.495, RMSE = 444.452
Testing  set : R2 = 0.497, RMSE = 438.429


Avec Alpha = 100.0
β1 de distance_km : 139.454
β1 de pickup_dayofweek : -13.514
β0 (intercept_) : 408.548

Training set : R2 = 0.495, RMSE = 444.452
Testing  set : R2 = 0.497, RMSE = 438.429




# Modèle 3

In [15]:
y = df.trip_duration
df_features = [
    "distance_km",
    "pickup_part4h"
]
X = df[df_features]

## Évaluation du modèle
### LinearRegression()

In [16]:
get_model_metrics(LinearRegression(), X, y)

β1 de distance_km : 139.576
β1 de pickup_part4h : 18.877
β0 (intercept_) : 309.934

Training set : R2 = 0.495, RMSE = 444.243
Testing  set : R2 = 0.498, RMSE = 438.246


### Lasso() avec différents alpha

In [17]:
for a in alphas:
    print(f"Avec Alpha = {a}")
    get_model_metrics(Lasso(alpha=a), X, y)
    print("\n")


Avec Alpha = 0.1
β1 de distance_km : 139.565
β1 de pickup_part4h : 18.836
β0 (intercept_) : 310.091

Training set : R2 = 0.495, RMSE = 444.243
Testing  set : R2 = 0.498, RMSE = 438.246


Avec Alpha = 1.0
β1 de distance_km : 139.47
β1 de pickup_part4h : 18.468
β0 (intercept_) : 311.501

Training set : R2 = 0.495, RMSE = 444.244
Testing  set : R2 = 0.498, RMSE = 438.244


Avec Alpha = 10.0
β1 de distance_km : 138.517
β1 de pickup_part4h : 14.788
β0 (intercept_) : 325.596

Training set : R2 = 0.495, RMSE = 444.301
Testing  set : R2 = 0.498, RMSE = 438.279


Avec Alpha = 100.0
β1 de distance_km : 129.265
β1 de pickup_part4h : 0.0
β0 (intercept_) : 399.222

Training set : R2 = 0.49, RMSE = 446.365
Testing  set : R2 = 0.493, RMSE = 440.188




### Ridge() avec différents alpha

In [18]:
for a in alphas:
    print(f"Avec Alpha = {a}")
    get_model_metrics(Ridge(alpha=a), X, y)
    print("\n")

Avec Alpha = 0.1
β1 de distance_km : 139.576
β1 de pickup_part4h : 18.877
β0 (intercept_) : 309.934

Training set : R2 = 0.495, RMSE = 444.243
Testing  set : R2 = 0.498, RMSE = 438.246


Avec Alpha = 1.0
β1 de distance_km : 139.576
β1 de pickup_part4h : 18.877
β0 (intercept_) : 309.935

Training set : R2 = 0.495, RMSE = 444.243
Testing  set : R2 = 0.498, RMSE = 438.246


Avec Alpha = 10.0
β1 de distance_km : 139.576
β1 de pickup_part4h : 18.877
β0 (intercept_) : 309.935

Training set : R2 = 0.495, RMSE = 444.243
Testing  set : R2 = 0.498, RMSE = 438.246


Avec Alpha = 100.0
β1 de distance_km : 139.575
β1 de pickup_part4h : 18.876
β0 (intercept_) : 309.941

Training set : R2 = 0.495, RMSE = 444.243
Testing  set : R2 = 0.498, RMSE = 438.246




# Modèle 4

In [19]:
y = df.trip_duration
df_features = [
    "pickup_dayofweek",
    "pickup_part4h"
]
X = df[df_features]

## Évaluation du modèle
### LinearRegression()

In [20]:
get_model_metrics(LinearRegression(), X, y)

β1 de pickup_dayofweek : -9.686
β1 de pickup_part4h : 10.902
β0 (intercept_) : 800.502

Training set : R2 = 0.002, RMSE = 624.685
Testing  set : R2 = 0.002, RMSE = 617.928


### Lasso() avec différents alpha

In [21]:
for a in alphas:
    print(f"Avec Alpha = {a}")
    get_model_metrics(Lasso(alpha=a), X, y)
    print("\n")


Avec Alpha = 0.1
β1 de pickup_dayofweek : -9.662
β1 de pickup_part4h : 10.864
β0 (intercept_) : 800.544

Training set : R2 = 0.002, RMSE = 624.685
Testing  set : R2 = 0.002, RMSE = 617.928


Avec Alpha = 1.0
β1 de pickup_dayofweek : -9.45
β1 de pickup_part4h : 10.523
β0 (intercept_) : 800.925

Training set : R2 = 0.002, RMSE = 624.686
Testing  set : R2 = 0.002, RMSE = 617.928


Avec Alpha = 10.0
β1 de pickup_dayofweek : -7.323
β1 de pickup_part4h : 7.115
β0 (intercept_) : 804.73

Training set : R2 = 0.002, RMSE = 624.734
Testing  set : R2 = 0.002, RMSE = 617.973


Avec Alpha = 100.0
β1 de pickup_dayofweek : -0.0
β1 de pickup_part4h : 0.0
β0 (intercept_) : 803.882

Training set : R2 = 0.0, RMSE = 625.251
Testing  set : R2 = -0.0, RMSE = 618.488




### Ridge() avec différents alpha

In [22]:
for a in alphas:
    print(f"Avec Alpha = {a}")
    get_model_metrics(Ridge(alpha=a), X, y)
    print("\n")

Avec Alpha = 0.1
β1 de pickup_dayofweek : -9.686
β1 de pickup_part4h : 10.902
β0 (intercept_) : 800.502

Training set : R2 = 0.002, RMSE = 624.685
Testing  set : R2 = 0.002, RMSE = 617.928


Avec Alpha = 1.0
β1 de pickup_dayofweek : -9.686
β1 de pickup_part4h : 10.902
β0 (intercept_) : 800.502

Training set : R2 = 0.002, RMSE = 624.685
Testing  set : R2 = 0.002, RMSE = 617.928


Avec Alpha = 10.0
β1 de pickup_dayofweek : -9.686
β1 de pickup_part4h : 10.902
β0 (intercept_) : 800.502

Training set : R2 = 0.002, RMSE = 624.685
Testing  set : R2 = 0.002, RMSE = 617.928


Avec Alpha = 100.0
β1 de pickup_dayofweek : -9.686
β1 de pickup_part4h : 10.901
β0 (intercept_) : 800.502

Training set : R2 = 0.002, RMSE = 624.685
Testing  set : R2 = 0.002, RMSE = 617.928




## Conclusion

- On remarque que seul la distance a vraiment des répercutions sur la distance.
- Cela peut-être dut au fait que les distances sont calculés à vol d'oiseau.
- De manière générale, avec un alpha supérieur à 10 le modèle devient moins fiable.